# FBA and pFBA Simulations

By Christina Schenk and Garrett Roell

Tested on biodesign_3.7 kernel on jprime


This notebook performs flux balance analysis (FBA) and parsimonious FBA for R. opacus in glucose and phenol growth conditions and creates plots to compare the results with experimental data.

### Methods: 
<ol>
<li>Calculate FBA and pFBA growth rates for each of the experimental conditions</li>
<li>Plot growth rates in comparison to experimental data</li>
<li>Compare phenol 13CMFA fluxes to FBA and pFBA predictions: Scatter plots and flux maps</li>
<li>Compare glucose 13CMFA fluxes to FBA and pFBA predictions: Scatter plots and flux maps</li>
</ol>

#### Imports

In [1]:
import pandas as pd
import numpy as np
import cobra
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.cbook import get_sample_data
import matplotlib.image as mpimg
import matplotlib.cm as cm
import scipy
from scipy.stats import linregress
from sklearn.metrics import r2_score

### Load data from consumption and growth rate notebook (notebook E)
Note: WT-P = wildtype grown with phenol, PVHG-P	= aromatic adaptive evolution strain grown with phenol, WT-LN-G = wildtype grown with phenol and low nitrogen

In [2]:
consumption_and_growth_data = pd.read_csv('../consumption_and_growth_data/consumption_and_growth_data_new.csv', index_col=0)
consumption_and_growth_data

,growth rate,yield coefficient,substrate consumption rate,growth rate std dev,yield coefficient std dev,substrate consumption rate std dev
WT-P,0.065488,0.048868,1.352072,0.001345,0.004836,0.122279
PVHG-P,0.080331,0.040848,1.967485,0.002672,0.001757,0.021785
WT-LN-G,0.260195,0.072789,3.582471,0.004749,0.004117,0.141675


#### Load model

In [3]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated.xml")

#### Calculate FBA fluxes for phenol

In [19]:
with model:
    # ensure that glucose will be limiting resource by setting all media components to 1000
    medium = {k: 1000 for k,v in model.medium.items()}
    
    # set glucose uptake rate from table (note: unit is mmol/g dry cell weight/hr)
    medium['EX_phenol_e'] = 1.352072
    
    # remove all non-glucose carbon sources
    medium['EX_glc__D_e'] = 0
    medium['EX_guaiacol_e'] = 0
    medium['EX_vanlt_e'] = 0
    medium['EX_tag'] = 0
    
    # set the model medium to the customized medium dictionary
    model.medium = medium
    
    # check medium
#     print(medium)
    
    # set objective function to maximize the glucose biomass reaction
    model.objective = 'Growth_Phenol'
    
    # fix the other biomass reactions to zero
    model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
    model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # run FBA to find the fluxes that maximize biomass production
    phenol_FBA_fluxes = model.optimize().fluxes
    phenol_FBA_growth_rate =  model.optimize().objective_value

print(f'The max FBA growth rate for glucose is {phenol_FBA_growth_rate:.5f} hr^-1')    

The max FBA growth rate for glucose is 0.12634 hr^-1


#### Save phenol FBA fluxes as csv for further analysis

In [5]:
# filter out low flux reactions
phenol_FBA_fluxes = phenol_FBA_fluxes[abs(phenol_FBA_fluxes) > 0.01]

# order the reactions by absolute value of flux
phenol_FBA_fluxes_abs = phenol_FBA_fluxes.map(lambda x : x).abs().sort_values(ascending = False)

# make dictionary to hold dictionary of reactions
phenol_FBA_fluxes_sorted = {}
for reaction in phenol_FBA_fluxes_abs.index:
    phenol_FBA_fluxes_sorted[reaction] = {
        'reaction': model.reactions.get_by_id(reaction).reaction,
        'name': model.reactions.get_by_id(reaction).name,
        'flux': phenol_FBA_fluxes[reaction],
    }
    
pd.DataFrame(phenol_FBA_fluxes_sorted).T.to_csv('../fba_data/phenol.csv')

#### Calculate FBA fluxes for glucose

In [18]:
with model:
    # ensure that glucose will be limiting resource by setting all media components to 1000
    medium = {k: 1000 for k,v in model.medium.items()}
    
    # set glucose uptake rate from table (note: unit is mmol/g dry cell weight/hr)
    medium['EX_glc__D_e'] = 3.582471
    
    # remove all non-glucose carbon sources
    medium['EX_phenol_e'] = 0
    medium['EX_guaiacol_e'] = 0
    medium['EX_vanlt_e'] = 0
    medium['EX_tag'] = 0
    
    # set the model medium to the customized medium dictionary
    model.medium = medium
    
    # check medium
#     print(medium)
    
    # set objective function to maximize the glucose biomass reaction
    model.objective = 'Growth_Glucose'
    
    # fix the other biomass reactions to zero
    model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
    model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # run FBA to find the fluxes that maximize biomass production
    glucose_FBA_fluxes = model.optimize().fluxes
    glucose_FBA_growth_rate =  model.optimize().objective_value

print(f'The max FBA growth rate for glucose is {glucose_FBA_growth_rate:.5f} hr^-1')

The max FBA growth rate for glucose is 0.43109 hr^-1


#### Save glucose FBA fluxes as csv for further analysis

In [7]:
# filter out low flux reactions
glucose_FBA_fluxes = glucose_FBA_fluxes[abs(glucose_FBA_fluxes) > 0.01]

# order the reactions by absolute value of flux
glucose_FBA_fluxes_abs = glucose_FBA_fluxes.map(lambda x : x).abs().sort_values(ascending = False)

# make dictionary to hold dictionary of reactions
glucose_FBA_fluxes_sorted = {}
for reaction in glucose_FBA_fluxes_abs.index:
    glucose_FBA_fluxes_sorted[reaction] = {
        'reaction': model.reactions.get_by_id(reaction).reaction,
        'name': model.reactions.get_by_id(reaction).name,
        'flux': glucose_FBA_fluxes[reaction],
    }
    
pd.DataFrame(glucose_FBA_fluxes_sorted).T.to_csv('../fba_data/glucose.csv')

#### Calculate Glucose FBA fluxes when EMP pathway and ED pathway is blocked

In [17]:
with model:
    # ensure that glucose will be limiting resource by setting all media components to 1000
    medium = {k: 1000 for k,v in model.medium.items()}
    
    # set glucose uptake rate from table (note: unit is mmol/g dry cell weight/hr)
    medium['EX_glc__D_e'] = 3.582471
    
    # remove all non-glucose carbon sources
    medium['EX_phenol_e'] = 0
    medium['EX_guaiacol_e'] = 0
    medium['EX_vanlt_e'] = 0
    medium['EX_tag'] = 0
    
    # set the model medium to the customized medium dictionary
    model.medium = medium
    
    # check medium
#     print(medium)
    
    # set objective function to maximize the glucose biomass reaction
    model.objective = 'Growth_Glucose'
    
    # fix the other biomass reactions to zero
    model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
    model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
    model.reactions.get_by_id('Growth').upper_bound = 0
    model.reactions.get_by_id('Growth').lower_bound = 0
    
    # block the EMP pathway to force ED flux (PGI is the reaction of G6P -> F6P
    model.reactions.get_by_id('PGI').upper_bound = 0
    model.reactions.get_by_id('PGI').lower_bound = 0
    
    # block the PP pathway
    model.reactions.get_by_id('GND').upper_bound = 0
    model.reactions.get_by_id('GND').lower_bound = 0
    
    # run FBA to find the fluxes that maximize biomass production
    glucose_FBA_fluxes = model.optimize().fluxes
    glucose_FBA_growth_rate =  model.optimize().objective_value


print(f'The max FBA growth rate for glucose when ED pathway is used is {glucose_FBA_growth_rate:.5f} hr^-1')

The max FBA growth rate for glucose when ED pathway is used is 0.41147 hr^-1


#### Save glucose FBA fluxes as csv for further analysis

In [9]:
# filter out low flux reactions
glucose_FBA_fluxes = glucose_FBA_fluxes[abs(glucose_FBA_fluxes) > 0.01]

# order the reactions by absolute value of flux
glucose_FBA_fluxes_abs = glucose_FBA_fluxes.map(lambda x : x).abs().sort_values(ascending = False)

# make dictionary to hold dictionary of reactions
glucose_FBA_fluxes_sorted = {}
for reaction in glucose_FBA_fluxes_abs.index:
    glucose_FBA_fluxes_sorted[reaction] = {
        'reaction': model.reactions.get_by_id(reaction).reaction,
        'name': model.reactions.get_by_id(reaction).name,
        'flux': glucose_FBA_fluxes[reaction],
    }
    
pd.DataFrame(glucose_FBA_fluxes_sorted).T.to_csv('../fba_data/glucose_emp_and_pp_blocked.csv')